good sources about this model :

https://youtu.be/A-yKQamf2Fc

https://dsgiitr.com/blogs/gat/

Can we do better than GCNs? 

From Graph Convolutional Network (GCN), we learnt that combining local graph structure and node-level features yields good performance on node classification task. However, the way GCN aggregates messages is structure-dependent, which may hurt its generalizability.


**GAT (Graph Attention Network)**, is a novel neural network architecture that operate on graph-structured data, leveraging masked self-attentional layers to address the shortcomings of prior methods based on graph convolutions or their approximations.
The fundamental novelty that GAT brings to the table is how the information from the one-hop neighborhood is aggregated.

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/Colab Notebooks/DMFOLDER/DATA

Mounted at /content/gdrive
/content/gdrive/MyDrive/Colab Notebooks/DMFOLDER/DATA


In [5]:
# Install required packages of pytorch geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu113.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
#then import needed ones
import pandas as pd
import sys
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import networkx as nx
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import StandardScaler
import os.path as osp
import matplotlib.pyplot as plt
import time
import warnings
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GNNExplainer

from sklearn.model_selection import train_test_split
from torch_geometric.data import InMemoryDataset
from sklearn.preprocessing import StandardScaler
from torch_geometric.utils import metric

     |████████████████████████████████| 7.9 MB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 5.4 MB/s 
     |████████████████████████████████| 145 kB 5.4 MB/s 
     |████████████████████████████████| 112 kB 44.0 MB/s 
     |████████████████████████████████| 74 kB 2.6 MB/s 
     |████████████████████████████████| 596 kB 49.0 MB/s 


In [ ]:
def filter_disease(disease_gene_association_file):
    columns = ['diseaseName', 'geneSymbol']
    df = pd.read_csv(disease_gene_association_file, sep='\t',usecols=columns)
    
    is_disease =  df['diseaseName']=="Schizophrenia" # first disease to be examined , comment this for checking for other disease and uncooment following
    #is_disease =  df['diseaseName']=="Asthma"
    df_disease = df[is_disease]
    return df_disease

In [ ]:
def create_G(biogrid_file_name,df_disease):
    biogrid = open(biogrid_file_name, 'r')
    lines=biogrid.readlines()
    G=nx.Graph()
    for line in tqdm(lines):
        x=line.split()
        if df_disease['geneSymbol'].str.contains(x[0]).any():
            G.add_node(x[0],label=1)
        else:
            G.add_node(x[0],label=0)
        if df_disease['geneSymbol'].str.contains(x[1]).any(): 
            G.add_node(x[1],label=1)
        else:  
            G.add_node(x[1],label=0)
        G.add_edge(x[0], x[1])
      #  G.add_edge(x[0], x[0]) # i decided to include an edge from node to itself to have adj matrix with himself included
       # G.add_edge(x[1], x[1])
    biogrid.close()
    G=nx.convert_node_labels_to_integers(G)
    #print(nx.info(G))
    return G

In [ ]:
def create_balanced_G(biogrid_file_name,df_disease):
    biogrid = open(biogrid_file_name, 'r')
    lines=biogrid.readlines()
    G=nx.Graph()
    ones=zeros=0
    i=0
    nums=[0]
    for line in tqdm(lines):
        x=line.split()
        if df_disease['geneSymbol'].str.contains(x[0]).any():
            G.add_node(x[0],label=1)
            ones=1+ones
        else :
          num=random.randint(0,100)
          if num in nums:
             G.add_node(x[0],label=0)
             zeros=zeros+1
             

        if df_disease['geneSymbol'].str.contains(x[1]).any(): 
            G.add_node(x[1],label=1)
            ones=1+ones
        else:
          num=random.randint(0,4000)
          if num in nums:
            G.add_node(x[1],label=0)
            zeros=1+zeros
        
        if G.has_node(x[0]) and G.has_node(x[1]):
            G.add_edge(x[0], x[1])
        i+=1
        if i%100000 ==0:
            #print("p = ",ones/(ones+zeros))
            #print(ones)
            #print(zeros)
            attribute_counter(G)
        

    biogrid.close()
    G=nx.convert_node_labels_to_integers(G)
    #print(nx.info(G))
    return G

In [7]:
def attribute_counter(G):
  zeros=0
  ones=0
  for node in tqdm(list(G.nodes())):
    if G.nodes[node]['label']==0:
      zeros+=1
    else:
      ones+=1
  print("#zeros: ", zeros)
  print("#ones: ", ones)
  print("portion of ones " ,ones/(ones+zeros) )

In [ ]:
print(" (0/6) started ")
#disease_gene_association_file="all_gene_disease_associations.tsv"  #current directory / this is real file not test file
disease_gene_association_file="curated_gene_disease_associations.tsv"
df_disease=filter_disease(disease_gene_association_file)
print(" (1/6) disease dataframe creation : done")
                   
#biogrid_file_name='Biogrid_REDUX.txt'
biogrid_file_name='Biogrid_4.4.199_Hs.txt'
G=create_G(biogrid_file_name,df_disease)
print(" (2/6) Graph creation : done")

In [ ]:
nx.write_gpickle(G, "Asthma_main.gpickle")

In [ ]:
G = nx.read_gpickle("Asthma_main.gpickle")

In [ ]:
print(nx.info(G))
print(G.nodes)
attribute_counter(G)

In [ ]:
balanced_G = create_balanced_G(biogrid_file_name,df_disease)

In [ ]:
attribute_counter(balanced_G)

In [ ]:
nx.write_gpickle(balanced_G, "Asthma_balanced_more.gpickle")

In [8]:
balanced_G = nx.read_gpickle("Asthma_balanced_more.gpickle")
nx.info(balanced_G)
attribute_counter(balanced_G)

100%|██████████| 5861/5861 [00:00<00:00, 113744.03it/s]

#zeros:  3681
#ones:  2180
portion of ones  0.37195017915031564


In [9]:
G=balanced_G

# Up to here everything is Graph prepraration and from now on is preparing data to be fed into NN. so instead of running all previous (time consuming steps , I load previously pickle saved models of graphs , for two diseases, for balanced and main version of graph derived from all_gene_disease_association ( or the other dataset)

In [10]:
# retrieve the labels for each node, the nodes that are not zero
labels = np.asarray([G.nodes[i]['label'] != 0 for i in G.nodes]).astype(np.int64)

# create edge index. We need to have data as previously shown. We can exploit networkX and scipy for that 
adj = nx.to_scipy_sparse_matrix(G).tocoo() #coordinate format
#print(adj)

In [11]:
#create edge index in the proper way
row = torch.from_numpy(adj.row.astype(np.int64)).to(torch.long) #create a torch tensor from numpy in long format : for row indexes
col = torch.from_numpy(adj.col.astype(np.int64)).to(torch.long) #                                                   for column indexes
edge_index = torch.stack([row, col], dim=0)
#display(edge_index)

In [12]:
# using degree as embedding. For simplicity, the feature vector describing the 
# will be just its degree, which is enough for us   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>.. can i use other embeddings? https://medium.com/@st3llasia/graph-embedding-techniques-7d5386c88c5
#is it actually working or we do it for simplicity
embeddings = np.array(list(dict(G.degree()).values())) #list the values of degree of each node as numpy array
# normalizing degree values

scale = StandardScaler()
embeddings = scale.fit_transform(embeddings.reshape(-1,1))
print(embeddings)

[[ 0.08454254]
 [-0.31812753]
 [-0.23271266]
 ...
 [-0.06188294]
 [-0.61097848]
 [ 1.79283978]]


In [13]:
# custom pytorch dataset
class PPIDATASET(InMemoryDataset):
    def __init__(self, transform=None):
        super(PPIDATASET, self).__init__('.', transform, None, None) #pre transform and pre filter: None, we don't need them
        data = Data(edge_index=edge_index) #Data : A data object describing a homogeneous graph.  for more : https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#torch_geometric.data.Data
        data.num_nodes = G.number_of_nodes()
        # embedding 
        data.x = torch.from_numpy(embeddings).type(torch.float32)
        # labels
        y = torch.from_numpy(labels).type(torch.long)
        data.y = y.clone().detach() #removing tensors computational graph for efficency since it is not needed
        data.num_classes = 2
        # splitting the data into train, validation and test
        X_train, X_test, y_train, y_test = train_test_split(pd.Series(G.nodes()),  pd.Series(labels), test_size=0.30, random_state=42)
        n_nodes = G.number_of_nodes()
        # create train and test masks for data
        # the Data objects holds a label for each node, and additional node-level attributes: train_mask, val_mask and test_mask, where
        #train_mask denotes against which nodes to train (140 nodes),
        #val_mask denotes which nodes to use for validation, e.g., t
        #test_mask denotes against which nodes to test 
        train_mask = torch.zeros(n_nodes, dtype=torch.bool)
        test_mask = torch.zeros(n_nodes, dtype=torch.bool)
        train_mask[X_train.index] = True
        test_mask[X_test.index] = True
        data['train_mask'] = train_mask
        data['test_mask'] = test_mask
        self.data, self.slices = self.collate([data])
    # def _download(self):
    #     return
    # def _process(self):
    #     return
    # def __repr__(self):
    #     return '{}()'.format(self.__class__.__name__)

In [14]:
dataset = PPIDATASET()
#Here, the dataset contains only a single, undirected citation graph, reminder: dataset is like a dictionary that hold the graph inside, here the dictionary
#has only one elemetns:
data = dataset[0] #now data is ready for training and testing

In [15]:
# Model Definition
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8     #input channels
        self.in_head = 8 #output channels
        self.out_head = 1 #output head
        #suggestion for later ::::: concatanation:true: for intermadiate layers
        self.conv1 = GATConv(dataset.num_features, self.hid, heads=self.in_head, dropout=0.7)
        self.conv2 = GATConv(self.hid*self.in_head, dataset.num_classes, concat=False, heads=self.out_head, dropout=0.7)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [16]:
# Seed for reproducible numbers
torch.manual_seed(1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

it is so fast on gpu that it becomes difficult to compare the models. if ran on cpu, it can be seen GAT is slower, for the same graph

In [17]:
# Train
model = GAT().to(device)
data = dataset[0].to(device)

# Adam Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.5, weight_decay=5e-4)

# Training Loop
model.train()
tick=time.time()
for epoch in tqdm(range(20)):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])    
    
    if epoch%2 == 0:
        print(loss)
    loss.backward()
    optimizer.step()
tock=time.time()


  5%|▌         | 1/20 [00:00<00:05,  3.46it/s]

tensor(2.7494, device='cuda:0', grad_fn=<NllLossBackward0>)


 10%|█         | 2/20 [00:01<00:14,  1.26it/s]

tensor(8.4588, device='cuda:0', grad_fn=<NllLossBackward0>)


 50%|█████     | 10/20 [00:02<00:01,  6.30it/s]

tensor(9.8140, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1351.0387, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(85.5875, device='cuda:0', grad_fn=<NllLossBackward0>)


 65%|██████▌   | 13/20 [00:02<00:00,  9.01it/s]

tensor(22.2186, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(22.0784, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(27.7647, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(23.6056, device='cuda:0', grad_fn=<NllLossBackward0>)


100%|██████████| 20/20 [00:02<00:00,  6.96it/s]

tensor(34.3958, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:

model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
train_acc = metric.accuracy(pred[data.test_mask],data.y[data.test_mask])
train_precision=metric.precision(pred[data.test_mask],data.y[data.test_mask],2)
train_f1score = metric.f1_score(pred[data.test_mask],data.y[data.test_mask],2)
train_recall = metric.recall(pred[data.test_mask],data.y[data.test_mask],2)
TP=metric.true_positive(pred[data.test_mask],data.y[data.test_mask],2)
TN= metric.true_negative(pred[data.test_mask],data.y[data.test_mask],2)
FN= metric.false_negative(pred[data.test_mask],data.y[data.test_mask],2)
FP= metric.false_positive(pred[data.test_mask],data.y[data.test_mask],2)
print("training done in ",(tock-tick),"seconds")
print('test Accuracy: %s' % train_acc)
print('test precision: %s' % list(train_precision)[1].item())
print('test f1 score: %s' % list(train_f1score)[1].item())
print('test recall: %s' % list(train_recall)[1].item())
print(" #### confusion matrix test: ")
print( "TP",list(TP)[1].item(),"FP",list(FP)[1].item())
print( "TN",list(TN)[1].item(),"FN",list(FN)[1].item())

the results for 20 epochs for Asthma (slower than gcn)

training done in  111.46260857582092 seconds

test Accuracy: 0.18409680207433016

test precision: 0.10513019561767578

test f1 score: 0.18648740649223328

test recall: 0.8246951103210449

 #### confusion matrix test: 

TP 541 FP 4605

TN 524 FN 115

can we do better with more epochs? ( with 50 epochs )

training done in  283.5730781555176 seconds

test Accuracy: 0.8866032843560934 accuracy increased

test precision: 0.0

test f1 score: 0.0

test recall: 0.0

 #### confusion matrix test: 

TP 0 FP 0

TN 5129 FN 656
 

# **for balanced version for Asthma**


training done in  31.540491819381714 seconds (more or less same time, not better/worst results) >  it doesnt mean it is worst than GCN ! because in GCN results with another initializations can be more or less (accuracy between 0.35 to 0.45) 


**test Accuracy:** 0.3854462762933485   > lower than gcn

**test precision** : 0.3854462802410126

**test f1 score**: 0.5564218163490295    > higher than gcn > 

**test recall**: 1.0

 #### confusion matrix test: 
TP 678 FP 1081

TN 0 FN 0

so maybe in my opinion i can say in term of metrics this is better than GCN

In [19]:
# Model Definition
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8     #input channels
        self.in_head = 8 #output channels
        self.out_head = 1 #output head
        #suggestion for later ::::: concatanation:true: for intermadiate layers
        self.conv1 = GATConv(dataset.num_features, self.hid, heads=self.in_head, dropout=0.7)
        self.conv2 = GATConv(self.hid*self.in_head, dataset.num_classes, concat=False, heads=self.out_head, dropout=0.7)

    def forward(self, x, edge_index, edge_weight):
        x, edge_index = data.x, data.edge_index
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.7, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [20]:
torch.manual_seed(42)

class Net(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_features, 16, normalize=False)
        self.conv2 = GCNConv(16, dataset.num_classes, normalize=False)

    def forward(self, x, edge_index, edge_weight):
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=1)

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GAT().to(device)
data = data.to(device)
lr = 1e-1

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
epochs = 200

x, edge_index, edge_weight = data.x, data.edge_index, None

num_epochs = 200

for epoch in tqdm(range(num_epochs)):
    model.train()
    optimizer.zero_grad()
    log_logits = model(x, edge_index, edge_weight)
    loss = F.nll_loss(log_logits[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

with torch.no_grad():
  model.eval()
  logits = model(x, edge_index, edge_weight)

  #test_mask = data['test_mask']
  preds = logits.max(1)[1]

100%|██████████| 200/200 [00:05<00:00, 37.41it/s]


In [29]:
explainer = GNNExplainer(model, epochs=200, return_type='log_prob', num_hops = None) #if num_hops is none it is detected from the num of message passing ops.
                                                                                  #it is needed to tell the explainer "how far to go" to look for explanations.
node_idx = 11 
pred_node_idx = preds[node_idx].item()
print("Explaining node ", node_idx, " with predicted class: ", pred_node_idx)
node_feat_mask, edge_mask = explainer.explain_node(node_idx, x, edge_index, edge_weight=edge_weight)


Explaining node  11  with predicted class:  1


In [ ]:
ax, G = explainer.visualize_subgraph(node_idx, edge_index, edge_mask, y=data.y, seed = node_idx, threshold = None) #you can set threshold to define the hard mask accordign to the sparisty you wnat ot obtain and how much you want to be strict
plt.show()